In [37]:
import psycopg2
from tabulate import tabulate
import html;

con = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="Postgre@1234")

#For isolation: SERIALIZABLE
con.set_isolation_level(3)
#For atomicity
con.autocommit = False

try:
    cur = con.cursor()
    # QUERY
    #Describe Table Schema
       
    cur.execute("SELECT ic.table_name, ic.column_name, ic.data_type FROM information_schema.columns ic where upper(ic.table_name) in ('PRODUCT','DEPOT','STOCK') order by ic.table_name;")
    print("Table Schema: ", cur.rowcount,"\n----------------")
    row = cur.fetchone()

    while row is not None:
        print(row)
        row = cur.fetchone()
        
    #Describe Table Constraints
    print("\n")   
    cur.execute("""SELECT ic.table_name, ic.column_name, ic.data_type, itc.constraint_name, itc.constraint_type 
                    FROM information_schema.columns ic 
                    join  information_schema.table_constraints itc 
                    on  ic.table_name = itc.table_name 
                    join information_schema.key_column_usage ik 
                    on ic.table_name = ik.table_name 
                    and ic.column_name = ik.column_name 
                    and itc.constraint_name = ik.constraint_name 
                    and upper(ic.table_name) in ('PRODUCT','DEPOT','STOCK') 
                    and itc.constraint_type <> 'CHECK' order by ic.table_name;""")
    print("Table Constraints: ", cur.rowcount,"\n--------------------")
    row = cur.fetchone()

    while row is not None:
        print(row)
        row = cur.fetchone()

    #Display Product Data
    print("\n")    
    cur.execute("select * from product;")
    print("The data from Product Table: ", cur.rowcount,"\n-----------------------------")
    row = cur.fetchone()

    while row is not None:
        print(row)
        row = cur.fetchone()

    # Display Depot Depot
    print("\n") 
    cur.execute("SELECT * FROM Depot;")
    print("The data from Depot Table: ", cur.rowcount,"\n--------------------------------")
    row = cur.fetchone()

    while row is not None:
        print(row)
        row = cur.fetchone()
        
    # Display Depot Stock
    print("\n") 
    cur.execute("SELECT * FROM STOCK;")
    print("The data from Stock Table: ", cur.rowcount,"\n--------------------------------")
    row = cur.fetchone()

    while row is not None:
        print(row)
        row = cur.fetchone()


    # If we try executing update statement on product or stock to change product_id p1 to pp1 we get following error 
    #postgres=# update product set prod_id = 'pp1' where prod_id = 'p1';
    #ERROR:  update or delete on table "product" violates foreign key constraint "fk_stock_prod" on table "stock"
    #DETAIL:  Key (prod_id)=(p1) is still referenced from table "stock".
    #postgres=# update stock set prod_id = 'pp1' where prod_id = 'p1';
    #ERROR:  insert or update on table "stock" violates foreign key constraint "fk_stock_prod"
    #DETAIL:  Key (prod_id)=(pp1) is not present in table "product".
    
    #To avoid above error we must follow ACID property 
    
    cur.execute("""alter table stock drop constraint fk_stock_prod;
                   alter table stock  add constraint fk_stock_prod foreign key (prod_id) 
                   references product (prod_id) on update cascade;
                   update product set prod_id = 'pp1' where prod_id = 'p1'; 
                    """)
    
        #Describe Table Constraints
    print("\n")   
    cur.execute("""SELECT ic.table_name, ic.column_name, ic.data_type, itc.constraint_name, itc.constraint_type 
                    FROM information_schema.columns ic 
                    join  information_schema.table_constraints itc 
                    on  ic.table_name = itc.table_name 
                    join information_schema.key_column_usage ik 
                    on ic.table_name = ik.table_name 
                    and ic.column_name = ik.column_name 
                    and itc.constraint_name = ik.constraint_name 
                    and upper(ic.table_name) in ('PRODUCT','DEPOT','STOCK') 
                    and itc.constraint_type <> 'CHECK' order by ic.table_name;""")
    print("Table Constraints: ", cur.rowcount,"\n--------------------")
    row = cur.fetchone()

    while row is not None:
        print(row)
        row = cur.fetchone()

    #Display Product Data
    print("\n")    
    cur.execute("select * from product;")
    print("The data from Product Table: ", cur.rowcount,"\n------------------------------")
    row = cur.fetchone()

    while row is not None:
        print(row)
        row = cur.fetchone()

    # Display Depot Depot
    print("\n") 
    cur.execute("SELECT * FROM Depot;")
    print("The data from Depot Table: ", cur.rowcount,"\n--------------------------------")
    row = cur.fetchone()

    while row is not None:
        print(row)
        row = cur.fetchone()
        
    # Display Depot Stock
    print("\n") 
    cur.execute("SELECT * FROM STOCK;")
    print("The data from Stock Table: ", cur.rowcount,"\n---------------------------------")
    row = cur.fetchone()

    while row is not None:
        print(row)
        row = cur.fetchone()
    
    cur.close()

except (Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Transactions could not be completed so database will be rolled back before start of transactions")
    con.rollback()
finally:
    if con:
        con.commit()
        print("\nTransaction Successfully committed")
        cur.close
        con.close
        print("\nPostgreSQL connection is now closed")

Table Schema:  9 
----------------
('depot', 'addr', 'character varying')
('depot', 'dep_id', 'character varying')
('depot', 'volumn', 'integer')
('product', 'price', 'double precision')
('product', 'prod_id', 'character varying')
('product', 'p_name', 'character varying')
('stock', 'dep_id', 'character varying')
('stock', 'prod_id', 'character varying')
('stock', 'quantity', 'integer')


Table Constraints:  6 
--------------------
('depot', 'dep_id', 'character varying', 'pk_depot', 'PRIMARY KEY')
('product', 'prod_id', 'character varying', 'pk_product', 'PRIMARY KEY')
('stock', 'dep_id', 'character varying', 'fk_stock_dep', 'FOREIGN KEY')
('stock', 'prod_id', 'character varying', 'fk_stock_prod', 'FOREIGN KEY')
('stock', 'dep_id', 'character varying', 'pk_stock', 'PRIMARY KEY')
('stock', 'prod_id', 'character varying', 'pk_stock', 'PRIMARY KEY')


The data from Product Table:  3 
-----------------------------
('p2', 'tv', 250.0)
('p3', 'vcr', 80.0)
('p1', 'tape', 2.5)


The data from

In [4]:
conda install psycopg2

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /Users/indirabadra/Anaconda/anaconda3

  added / updated specs:
    - psycopg2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.25  |       hecd8cb5_1         111 KB
    certifi-2021.5.30          |   py38hecd8cb5_0         139 KB
    conda-4.10.3               |   py38hecd8cb5_0         2.9 MB
    curl-7.69.1                |       ha441bb4_0         127 KB
    krb5-1.17.1                |       hddcf347_0         1.1 MB
    libcurl-7.69.1             |       h051b688_0         382 KB
    libpq-12.2                 |       h051b688_0         2.0 MB
    psycopg2-2.8.6             |   py38hbcfaee0_1       

In [6]:
conda install -c conda-forge tabulate

Solving environment: done

## Package Plan ##

  environment location: /Users/indirabadra/Anaconda/anaconda3

  added / updated specs:
    - tabulate


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.30  |       h033912b_0         136 KB  conda-forge
    certifi-2021.5.30          |   py38h50d1736_0         141 KB  conda-forge
    conda-4.10.3               |   py38h50d1736_0         3.1 MB  conda-forge
    python_abi-3.8             |           2_cp38           4 KB  conda-forge
    tabulate-0.8.9             |     pyhd8ed1ab_0          26 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.4 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/osx-64::python_abi-3.8-2_cp38
  tabulate           conda-forge/noarch::tabulate-0.8.9-pyhd8ed1ab_0

The